In [1]:
import json
import pandas as pd

# import numpy as np   
# import matplotlib.pyplot as plt
# %matplotlib inline
# import seaborn as sns
# import warnings
# warnings.filterwarnings('ignore') # To supress warnings
# sns.set(style="darkgrid") # set the background for the graphs
# pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
# obtém categorias dos vídeos a partir de arquivo json
# precisamos de "id" e "title" que são acessíveis por "items" e "snippet"."title"

with open("BR_category_id.json") as f:
    categories = json.load(f)["items"]

cat_dict = {'29': 'Nonprofits & Activism'} # insere categoria que está faltando manualmente
category = [['29', 'Nonprofits & Activism']]
for cat in categories:
    category.append([cat["id"],cat["snippet"]["title"]])
    cat_dict[cat["id"]] = cat["snippet"]["title"]
    
print(cat_dict)

def category_from_id(row):
    return cat_dict[str(row['categoryId'])]

# constroi Dataframe com o conteúdo das categorias (apenas para exibição)
#df_category = pd.DataFrame(category, columns=['categoryId','category']) 
#df_category

{'29': 'Nonprofits & Activism', '1': 'Film & Animation', '2': 'Autos & Vehicles', '10': 'Music', '15': 'Pets & Animals', '17': 'Sports', '18': 'Short Movies', '19': 'Travel & Events', '20': 'Gaming', '21': 'Videoblogging', '22': 'People & Blogs', '23': 'Comedy', '24': 'Entertainment', '25': 'News & Politics', '26': 'Howto & Style', '27': 'Education', '28': 'Science & Technology', '30': 'Movies', '31': 'Anime/Animation', '32': 'Action/Adventure', '33': 'Classics', '34': 'Comedy', '35': 'Documentary', '36': 'Drama', '37': 'Family', '38': 'Foreign', '39': 'Horror', '40': 'Sci-Fi/Fantasy', '41': 'Thriller', '42': 'Shorts', '43': 'Shows', '44': 'Trailers'}


In [3]:
# abre dataframe e mostra informações

df = pd.read_csv('BR_youtube_trending_data.csv')
print(f'Período: {df.publishedAt.min()} até {df.publishedAt.max()}')
#df.head()
#df.tail()

Período: 2020-08-06T10:08:27Z até 2021-08-27T21:00:01Z


In [4]:
# Adiciona, remove e renomeia colunas

df['max_view_count'] = df.groupby(['video_id'])['view_count'].transform(max)
df['category'] = df.apply(category_from_id, axis=1)

#df.rename(columns={'comment_count':'commentcount','thumbnail_link':'thumbnaillink'},inplace=True)
#df.drop('ratings_disabled',inplace=True,axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77593 entries, 0 to 77592
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           77593 non-null  object
 1   title              77593 non-null  object
 2   publishedAt        77593 non-null  object
 3   channelId          77593 non-null  object
 4   channelTitle       77593 non-null  object
 5   categoryId         77593 non-null  int64 
 6   trending_date      77593 non-null  object
 7   tags               77593 non-null  object
 8   view_count         77593 non-null  int64 
 9   likes              77593 non-null  int64 
 10  dislikes           77593 non-null  int64 
 11  comment_count      77593 non-null  int64 
 12  thumbnail_link     77593 non-null  object
 13  comments_disabled  77593 non-null  bool  
 14  ratings_disabled   77593 non-null  bool  
 15  description        75714 non-null  object
 16  max_view_count     77593 non-null  int64

In [ ]:
# Seleção de colunas

#colunas = ['channelId', 'title', 'publishedAt']
#df[colunas].head() #mostra apenas colunas da lista

#df.filter(regex='chan') #mostra apenas colunas que batem com a regex

In [ ]:
# Seleção de linhas

#df.nlargest(10,'view_count') # n= 10
#df.nsmallest(10,'view_count')
#df.sample(5)
#df.iloc[1:5]
df.loc[(df.channelTitle=='UOL Esporte')].count()
df.loc[(df.channelTitle=='UOL Esporte'),'channelId':'view_count'] # seleciona linhas e colunas
df.loc[:,'categoryId'].value_counts()
#df.query('video_id=="-0ZGKFwF1-4"') #pandas.core.frame.DataFrame
#df.loc[(df.video_id=='-0ZGKFwF1-4')] #pandas.core.frame.DataFrame
mask = (df.publishedAt.dt.date >= pd.to_datetime('2021-01-01')) & (df.publishedAt.dt.date <= pd.to_datetime('2021-01-31'))
df.loc[mask]

In [ ]:
# Agrupamentos e criação de Dataframes a partir de um existente

# agrupa por canal e mostra o total de vídeos de cada um
#df.groupby("channelTitle").size().reset_index(name="video_count") \
#    .sort_values("video_count", ascending=False).head(20)

# agrupa por categoria e aplica uma lista de funções (com nomes informados por strings) a uma coluna do grupo
#agg_func_count = {'view_count': ['min', 'max', 'count', 'nunique']}
#df.groupby(['category']).agg(agg_func_count)

Por período (data início e data fim):
- Canais com mais views (gráfico de barras)
- Vídeos com mais views (gráfico de barras)
- Relação entre views/comentários (scatter)
- Relação entre views/likes (scatter)

Tabela:
- Mostrar tudo (n vídeos)
- Listar categorias
- Listar canais
- Total de vídeos
- Mostrar apenas dados da categoria selecionada pelo usuário (radio button)
- Mostrar apenas dados do canal inserido pelo usuário
- Mostrar apenas dados da categoria/título do vídeo inserido pelo usuário
- Ordenar tabela por views de vídeos
- Ordenar tabela por likes de vídeos
- Ordenar tabela por comments de vídeos
- (Mesmo acima, mas por canal)

In [5]:
# Modificações no dataset original para adequação ao projeto

## altera tipo da coluna (não é salvo no novo arquivo)
df.publishedAt = pd.to_datetime(df.publishedAt)
df.trending_date = pd.to_datetime(df.trending_date)

## constroi dataframe apenas com vídeos únicos (inclui apenas última [max] view_count de cada vídeo)
df = df.loc[df.view_count == df.max_view_count]

## remove colunas que não serão usadas
df.drop('categoryId', inplace=True, axis=1)
df.drop('tags', inplace=True, axis=1)
df.drop('thumbnail_link', inplace=True, axis=1)
df.drop('comments_disabled', inplace=True, axis=1)
df.drop('ratings_disabled', inplace=True, axis=1)
df.drop('max_view_count', inplace=True, axis=1)

## renomeia colunas para o pt-br
df.rename(columns={'video_id': 'id_video', 'title': 'titulo',
                   'publishedAt': 'dt_publicacao',
                   'channelId': 'id_canal',
                   'channelTitle': 'canal',
                   'trending_date': 'dt_trending',
                   'view_count': 'cont_views',
                   'comment_count': 'cont_comentarios',
                   'description': 'descricao',
                   'max_view_count': 'max_cont_views',
                   'category': 'categoria'
                  }, inplace=True)

## cria colunas com ano e mês separados
#df['ano_pub'] = df['publishedAt'].dt.year
#df['mes_pub'] = df.publishedAt.dt.month
#df['dia_pub'] = df.publishedAt.dt.day

In [6]:
# Imprime informações e gera dataframes menores para testes

df.info()

dfsc1 = df.sample(20)
dfsc2 = df.sample(20)
dfsc3 = df.sample(20)

df.to_csv('BR_youtube_trending_data_completo.csv', encoding='utf-8', index=False)
#dfsc1.to_csv('BR_youtube_trending_data_p1.csv', encoding='utf-8', index=False)
#dfsc2.to_csv('BR_youtube_trending_data_p2.csv', encoding='utf-8', index=False)
#dfsc3.to_csv('BR_youtube_trending_data_p3.csv', encoding='utf-8', index=False)

#[can for can in zip(df['channelTitle'])]
#[row for row in zip(df['channelTitle'], df['title'], df['publishedAt'], df['category'])]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11156 entries, 13 to 77592
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   id_video          11156 non-null  object             
 1   titulo            11156 non-null  object             
 2   dt_publicacao     11156 non-null  datetime64[ns, UTC]
 3   id_canal          11156 non-null  object             
 4   canal             11156 non-null  object             
 5   dt_trending       11156 non-null  datetime64[ns, UTC]
 6   cont_views        11156 non-null  int64              
 7   likes             11156 non-null  int64              
 8   dislikes          11156 non-null  int64              
 9   cont_comentarios  11156 non-null  int64              
 10  descricao         10880 non-null  object             
 11  categoria         11156 non-null  object             
dtypes: datetime64[ns, UTC](2), int64(4), object(6)
memory usage

In [ ]:
ex_df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar','foo', 'bar'],
                   'B' : ['one', 'one', 'two', 'three','two', 'two'],
                   'C' : [1, 5, 5, 2, 5, 5],
                   'D' : [2.0, 5., 8., 1., 2., 9.]})
ex_df

In [ ]:
ex_df['total'] = ex_df.groupby(['A'])['C'].transform(sum)
#ex_df.groupby('A').apply(print)
ex_df

# Referências

- [Kaggle - YouTube Trending Video Dataset (updated daily)](https://www.kaggle.com/rsrishav/youtube-trending-video-dataset)
- [Pandas tutorial and cheat sheet](https://www.kaggle.com/yogidsba/pandas-tutorial-and-cheat-sheet)
- [Write a database class in Python](https://stackoverflow.com/questions/37408081/write-a-database-class-in-python/37408275)
- [Convert a Pandas DataFrame into a list of objects](https://stackoverflow.com/questions/53192602/convert-a-pandas-dataframe-into-a-list-of-objects)